In [1]:
from requests import get
from pandas import DataFrame, to_datetime

link = "https://api.epias.com.tr/epias/exchange/transparency/market/day-ahead-mcp"

### Selecting start and end dates

In [2]:
start_date = "2019-01-01"
end_date = "2023-06-30"

payload = "?startDate={}&endDate={}".format(start_date, end_date)

### Requesting data

In [3]:
request = get(link + payload)
result = request.json()

### Response

In [4]:
df = DataFrame(result["body"]["dayAheadMCPList"])
df.head()

,date,price,priceUsd,priceEur
0,2019-01-01T00:00:00.000+0300,100.38,19.01,16.61
1,2019-01-01T01:00:00.000+0300,96.72,18.31,16.01
2,2019-01-01T02:00:00.000+0300,81.60,15.45,13.51
3,2019-01-01T03:00:00.000+0300,38.58,7.31,6.39
4,2019-01-01T04:00:00.000+0300,11.52,2.18,1.91


### Formatting dataframe

In [5]:
df["date"] = to_datetime(df["date"])

df["day"] = df["date"].dt.dayofyear
df["year"] = df["date"].dt.year
df["week"] = df["date"].dt.weekday
df["month"] = df["date"].dt.month
df["dd"] = df["date"].dt.day

weeks = df.set_index(["year","day"])["week"].to_dict()
days = df.set_index(["year","day"])[["dd","month"]].to_dict()

key = df.groupby(["year","day"])["price"].mean().unstack().T
key = key.iloc[:-1,:]

df = key.stack().reorder_levels(["year","day"]).sort_index()
df = df.reset_index()
df.columns = ["y","d","ptf"]

df["r"] = 360*(df.d -1) / 365 + (df.y - 2019)*360
df["w"] = df.apply(lambda x: weeks[x["y"],x["d"]],axis = 1)

df["dayNames"] = df["w"].replace({0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",
                                5:"Saturday",6:"Sunday"})

df["hover"] = df.apply(lambda x: 
                       
                       "<br>Date: "+
                       "{}".format(days["dd"][x["y"],x["d"]]).zfill(2)+"-"+
                       "{}".format(days["month"][x["y"],x["d"]]).zfill(2)+"-"+
                       "{}".format(x["y"])+
                       "<br>Week Day: {}".format(x["dayNames"])+
                       "<br>Price: {:.2f}<br>".format(x["ptf"] ) +
                       "<extra></extra>"
                       ,
                       axis = 1)
df["i"] = [i for i in range(len(df))]


### Creating external JS file

In [6]:
with open("data.js", "w") as file:
    file.write("var data = ")
    file.write(df.to_json())
